In [3]:
import pandas as pd

In [7]:
orders = pd.read_csv(r"C:\Users\SUMANTH GAJJELA\Downloads\orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [67]:
users = pd.read_json(r"C:\Users\SUMANTH GAJJELA\Downloads\users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [73]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(":memory:")

with open(r"C:\Users\SUMANTH GAJJELA\Downloads\restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

conn.executescript(sql_script)

In [74]:
restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants_df

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


In [75]:
merged_data = merged_data.merge(
    restaurants_df,
    on="restaurant_id",
    how="left"
)

In [31]:
merged_data['restaurant_id'] = merged_data['restaurant_id'].astype(int)
restaurants['restaurant_id'] = restaurants['restaurant_id'].astype(int)

final_data = merged_data.merge(restaurants, on="restaurant_id", how="left")
final_data.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city_x,membership,restaurant_name_y,city_y
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,NaN,NaN
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,NaN,NaN
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,NaN,NaN
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,NaN,NaN
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,NaN,NaN


In [38]:
final_data.to_csv("final_food_delivery_dataset.csv", index=False)

In [76]:
gold_members_df = merged_data[merged_data["membership"] == "Gold"]
print(gold_members_df.groupby("city")["total_amount"].sum().sort_values(ascending=False))

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [77]:
merged_data.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [86]:
user_totals_df = (
    merged_data
    .groupby("user_id", as_index=False)["total_amount"]
    .sum()
)

int((user_totals_df["total_amount"] > 1000).sum())


2544

In [89]:
merged_data.groupby(
    pd.cut(merged_data["rating"], [3.0,3.6,4.1,4.6,5.0])
)["total_amount"].sum().sort_values(ascending=False)

C:\Users\SUMANTH GAJJELA\AppData\Local\Temp\ipykernel_3188\3009478334.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_data.groupby(


rating
(3.0, 3.6]    2145948.51
(4.1, 4.6]    2075343.38
(3.6, 4.1]    1834151.32
(4.6, 5.0]    1701162.78
Name: total_amount, dtype: float64

In [90]:
merged_data[merged_data["membership"]=="Gold"] \
.groupby("city")["total_amount"].mean().sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [91]:
merged_data.groupby("cuisine").agg(
    restaurant_count=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
).sort_values(["restaurant_count","revenue"], ascending=[True,False])

,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [92]:
round((len(merged_data[merged_data["membership"]=="Gold"]) / len(merged_data))*100)

50

In [95]:
merged_data.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range'],
      dtype='object')

In [96]:
restaurant_stats_df = merged_data.groupby("restaurant_name_x").agg(
    order_count=("order_id","count"),
    avg_order_value=("total_amount","mean")
)

restaurant_stats_df[restaurant_stats_df["order_count"] < 20] \
.sort_values("avg_order_value", ascending=False)

,order_count,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [97]:
merged_data.groupby(["membership","cuisine"])["total_amount"] \
.sum().sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [98]:
merged_data.groupby(
    pd.to_datetime(merged_data["order_date"]).dt.quarter
)["total_amount"].sum().sort_values(ascending=False)

C:\Users\SUMANTH GAJJELA\AppData\Local\Temp\ipykernel_3188\2428530950.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pd.to_datetime(merged_data["order_date"]).dt.quarter


order_date
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [99]:
round(merged_data[merged_data["city"]=="Hyderabad"]["total_amount"].sum())

1889367

In [100]:
merged_data["user_id"].nunique()

2883

In [105]:
distinct_users = merged_data["user_id"].nunique()
int(distinct_users)

2883

In [102]:
gold_orders = merged_data[merged_data["membership"] == "Gold"]
round(gold_orders["total_amount"].mean(), 2)

np.float64(797.15)

In [106]:
gold_orders = merged_data[merged_data["membership"] == "Gold"]
avg_order_value_gold = gold_orders["total_amount"].mean()
round(float(avg_order_value_gold), 2)

797.15

In [104]:
gold_orders = merged_data[merged_data["membership"] == "Gold"]
city_revenue = gold_orders.groupby("city")["total_amount"].sum()

top_city = city_revenue.idxmax()
len(gold_orders[gold_orders["city"] == top_city])

1337

In [ ]:
# MCQ Analysis – Cell-wise Explanation

This document explains **each analysis cell** used in the notebook and **what its output represents**. The explanations are written in clear, simple English.

---

## Cell 1 – Filter Gold Member Orders

**What we did:**
We filtered the dataset to keep only orders placed by Gold members.

**How it works:**
The `membership` column was checked, and only rows with value `Gold` were selected.

**Output meaning:**
A smaller dataset containing only Gold member orders, used in later questions.

---

## Cell 2 – City with Highest Revenue from Gold Members

**What we did:**
We grouped Gold member orders by city and calculated total revenue for each city.

**How it works:**

* Grouped by `city`
* Added `total_amount` values using `sum()`
* Sorted results in descending order

**Output meaning:**
The city shown at the top generated the highest revenue from Gold members.

---

## Cell 3 – Cuisine with Highest Average Order Value

**What we did:**
We calculated the average order value for each cuisine.

**How it works:**

* Grouped orders by `cuisine`
* Used `mean()` on `total_amount`
* Sorted results from highest to lowest

**Output meaning:**
The top cuisine indicates where customers spend the most per order on average.

---

## Cell 4 – Users Spending More Than ₹1000

**What we did:**
We calculated total spending per user and counted users whose total exceeded ₹1000.

**How it works:**

* Grouped `orders_df` by `user_id`
* Summed `total_amount`
* Filtered totals greater than ₹1000

**Output meaning:**
The number represents how many high-value users exist in the dataset.

---

## Cell 5 – Rating Range with Highest Revenue

**What we did:**
We divided restaurant ratings into ranges and calculated revenue for each range.

**How it works:**

* Converted ratings into ranges using `pd.cut()`
* Grouped by rating ranges
* Summed revenue

**Output meaning:**
The rating range at the top shows which quality level earns the most revenue.

---

## Cell 6 – City with Highest Average Order Value (Gold Members)

**What we did:**
We calculated average order value per city for Gold members.

**How it works:**

* Filtered Gold member orders
* Grouped by city
* Used `mean()`

**Output meaning:**
The top city shows where Gold members place higher-value orders.

---

## Cell 7 – Cuisine with Fewer Restaurants but High Revenue

**What we did:**
We compared the number of restaurants and revenue for each cuisine.

**How it works:**

* Counted unique restaurants per cuisine
* Calculated total revenue per cuisine
* Sorted by lowest restaurant count and high revenue

**Output meaning:**
This highlights cuisines that perform well despite having fewer restaurants.

---

## Cell 8 – Percentage of Orders from Gold Members

**What we did:**
We calculated what percentage of total orders were placed by Gold members.

**How it works:**

* Counted Gold member orders
* Divided by total orders
* Converted to percentage

**Output meaning:**
Shows the contribution of Gold members to overall order volume.

---

## Cell 9 – Restaurant with High Average Value and Few Orders

**What we did:**
We found restaurants with fewer than 20 orders but high average order value.

**How it works:**

* Grouped by restaurant name
* Calculated order count and average value
* Filtered restaurants with less than 20 orders

**Output meaning:**
The top restaurant is premium but less frequently ordered.

---

## Cell 10 – Highest Revenue Combination (Membership + Cuisine)

**What we did:**
We checked which membership and cuisine combination earns the most revenue.

**How it works:**

* Grouped by `membership` and `cuisine`
* Summed revenue

**Output meaning:**
The top combination shows the most profitable customer–cuisine pair.

---

## Cell 11 – Quarter with Highest Revenue

**What we did:**
We analyzed revenue across quarters of the year.

**How it works:**

* Converted order dates to quarters
* Grouped by quarter
* Summed revenue

**Output meaning:**
The top quarter represents peak business season.

---

## Cell 12 – Total Revenue from Hyderabad

**What we did:**
We calculated total revenue from orders placed in Hyderabad.

**How it works:**

* Filtered orders by city
* Summed `total_amount`

**Output meaning:**
Shows Hyderabad’s total contribution to revenue.

---

## Cell 13 – Total Distinct Users

**What we did:**
We counted how many unique users placed at least one order.

**How it works:**

* Used `nunique()` on `user_id`

**Output meaning:**
Represents total active users on the platform.

---

## Cell 14 – Average Order Value for Gold Members

**What we did:**
We calculated the average spending per order for Gold members.

**How it works:**

* Filtered Gold member orders
* Calculated mean order value

**Output meaning:**
Shows spending behavior of premium customers.

---

## Cell 15 – Orders in Top Revenue City (Gold Members)

**What we did:**
We found how many Gold member orders came from the highest revenue city.

**How it works:**

* Identified top revenue city among Gold members
* Counted orders from that city

**Output meaning:**
Shows order concentration in the most valuable city.

---

**End of Document**
